## Filter Data
Now we will filter our data. For this we'll merge our 3 dataframes. We will left join, because there are authors in the authors dataframe which do not have any title/chapter in the other dataframes.

In [1]:
import pandas as pd

In [2]:
chapters = pd.read_csv('data/chapters.csv')
authors = pd.read_csv('data/authors.csv')
titles = pd.read_csv('data/titles.csv')

In [3]:
df = (chapters.merge(titles, on='title_url', how='left')
      .merge(authors, on='author_url', how='left')
     )

In [4]:
df.head()

,title_url,chapter_num,chapter,author_url,title,year,author,born,died
0,/gellert/comoedia/comoedia.html,0,"Man hat zu unsern Zeiten, besonders in Frankre...",gellert,Pro comoedia commovente,1751,Christian Fürchtegott Gellert,1715,1769
1,/gellert/comoedia/comoedia.html,1,Ich komme nunmehr auf den zweiten Einwurf. Rüh...,gellert,Pro comoedia commovente,1751,Christian Fürchtegott Gellert,1715,1769
2,/gellert/comoedia/comoedia.html,2,Bei jeder Erdichtung nemlich verursacht nicht ...,gellert,Pro comoedia commovente,1751,Christian Fürchtegott Gellert,1715,1769
3,/gellert/zschwest/zschwest.html,0,"CleonDer Magister, sein BruderLottchen, Cleons...",gellert,Die zärtlichen Schwestern,1747,Christian Fürchtegott Gellert,1715,1769
4,/gellert/zschwest/zschwest.html,1,"Cleon. Lottchen.Lottchen. Lieber Papa, Herr Da...",gellert,Die zärtlichen Schwestern,1747,Christian Fürchtegott Gellert,1715,1769


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37313 entries, 0 to 37312
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title_url    37313 non-null  object
 1   chapter_num  37313 non-null  int64 
 2   chapter      36682 non-null  object
 3   author_url   37313 non-null  object
 4   title        37313 non-null  object
 5   year         36708 non-null  object
 6   author       37313 non-null  object
 7   born         37313 non-null  int64 
 8   died         37313 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 2.8+ MB


We see that some chapters do not contain any values. This might be due to some illustrations or different html-structures. 

In [6]:
df = df.loc[~df.chapter.isna()]

In [7]:
df.loc[df.year.isna()].head()

,title_url,chapter_num,chapter,author_url,title,year,author,born,died
6678,/diderot/rameaus/rameaus.html,0,Mit Zeichnungen von Antoine Watteau»Rameaus Ne...,goethe,Rameaus Neffe,NaN,Johann Wolfgang von Goethe,1749,1832
6679,/diderot/rameaus/rameaus.html,1,"Es mag schön oder häßlich Wetter sein, meine G...",goethe,Rameaus Neffe,NaN,Johann Wolfgang von Goethe,1749,1832
6680,/diderot/rameaus/rameaus.html,2,"Der Übersetzer hatte sich vorgenommen, die Per...",goethe,Rameaus Neffe,NaN,Johann Wolfgang von Goethe,1749,1832
6681,/diderot/rameaus/rameaus.html,3,Nachdem die französische Übersetzung des Dider...,goethe,Rameaus Neffe,NaN,Johann Wolfgang von Goethe,1749,1832
7217,/lenz/empfinds/empfinds.html,0,oder \n\n,lenz,Empfindsamster aller Romane,NaN,Jakob Lenz,1751,1792


Also some of the rows do not have a year-value. After looking into it, a possible reason is that the metadata for 'firstpub' exists, but only consists of an empty string. We can tweak the data collection code for this later.

In [8]:
df = df.loc[~df.year.isna()]

In some collections (often collections of poems) there are title pages where 'Projekt Gutenberg' is mentioned. This doesn't belong to the original book.

In [9]:
df = df.loc[~df.chapter.str.contains('Projekt Gutenberg')]

For the first iteration it is enough for us to consider books where the year of first publishment was already given in the metadata of the webpage. We assigned the values 0 (for missing firstpub-data / or data in the wrong format) and 2 (url status_code different from 200) to titles with missing years.

Because of some missing years, we first have to convert the year-data into integer values.

In [10]:
#df['year'] = pd.to_numeric(df['year'])

We are unable to do so, because some strings are of the form 
- '1800-1801' or
- '1778/79' or
- '1854/1855'. 
To keep things simple, we will just keep the first numbers (first 4 characters).

In [11]:
df.loc[df.year.str.len() >4].sample(50)

,title_url,chapter_num,chapter,author_url,title,year,author,born,died
7875,/herder/volklied/volklied.html,83,Französisch.Das erste Lied ist von Fenelon: da...,herder,Stimmen der Völker in Liedern,1778/79,Johann Gottfried Herder,1744,1803
7794,/herder/volklied/volklied.html,2,Deutsch.Aus dem Munde des Volks in Elsaß. Die ...,herder,Stimmen der Völker in Liedern,1778/79,Johann Gottfried Herder,1744,1803
31833,/freytag/dvergang/dvergang.html,52,(1790)Verschiedene Grundlagen für Sittlichkeit...,freytag,Bilder aus der deutschen Vergangenheit,1859-67,Gustav Freytag,1816,1895
7949,/herder/volklied/volklied.html,157,Spanisch.Obras de Gongora p. –Alle Töchter der...,herder,Stimmen der Völker in Liedern,1778/79,Johann Gottfried Herder,1744,1803
29223,/gutzkow/zauber2/zauber2.html,0,"Den Strom zu nennen, auf dem soeben ein in ers...",gutzkow,"Der Zauberer von Rom, 2. Buch",1858-61,Karl Gutzkow,1811,1878
27894,/gotthelf/geltstag/geltstag.html,9,"Wenn auf dem Meere der Sturm beginnt, eine sch...",gotthelf,Der Geltstag,1845/46,Jeremias Gotthelf,1797,1854
28281,/gotthelf/annebae2/annebae2.html,2,"Seltsam war es am Morgen Meyeli zumute, als es...",gotthelf,Wie Anne Bäbi Jowäger haushaltet und wie es ih...,1843/44,Jeremias Gotthelf,1797,1854
37036,/musil/mannohn3/mannohn3.html,31,Fühlen und Verhalten. Die Unsicherheit des Gef...,musil,Der Mann ohne Eigenschaften. Schluß des dritte...,1930 - 1932,Robert Musil,1880,1942
37019,/musil/mannohn3/mannohn3.html,14,"Gespräche über Liebe Der Mensch, recht eigentl...",musil,Der Mann ohne Eigenschaften. Schluß des dritte...,1930 - 1932,Robert Musil,1880,1942
29309,/gutzkow/zauber8/zauber8.html,2,Eines Winterabends herrschte auf Schloß Bex ei...,gutzkow,"Der Zauberer von Rom, 8. Buch",1858-61,Karl Gutzkow,1811,1878


In [12]:
df.loc[df.year.str.len() > 4, 'year'] = df.loc[df.year.str.len() > 4, 'year'].str[:4]
df['year'] = pd.to_numeric(df['year'])

In [13]:
df = df.loc[df.year > 10]

Unfortunately this was quite a chunk. But we can come back to the year in the second iteration.

As we only care for the first publishment year, we will only consider publishments when the author was still alive, i.e. 'born' < 'year' < 'died'. Otherwise it is too likely that we have a later edition or a collection of works.

In [14]:
df = df.loc[(df.born < df.year) & (df.year < df.died)]

Lastly, we check for duplicates that might have emerged from merging the dataframes.

In [15]:
df.duplicated().sum() #that hurt

9237

In [16]:
df.drop_duplicates(inplace=True)

In [17]:
cols = ['title', 'author', 'chapter', 'year', 'chapter_num', 'title_url', 'author_url', 'born', 'died']
df = df[cols]

In [18]:
df.duplicated(subset = 'chapter').sum()
df = df.drop_duplicates(subset='chapter')

In [19]:
df.to_csv('data/df_filtered.csv', index=False)